In [1]:
import pandas as pd
import numpy as np
dfactivity = pd.read_csv('D:/DATA ANALYSIS/Final_proj/dbases/activity.csv')
dfgroups = pd.read_csv('D:\DATA ANALYSIS\Final_proj\dbases\groups.csv')
dfusers = pd.read_csv('D:/DATA ANALYSIS/Final_proj/dbases/users.csv')

In [2]:
dfgroups = dfgroups.rename(columns={'group': 'group_name'})

dfactivity.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2233 entries, 0 to 2232
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   uid     2233 non-null   int64  
 1   dt      2233 non-null   object 
 2   device  2223 non-null   object 
 3   spent   2233 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 69.9+ KB


In [3]:
##Can a user show up more than once in the activity table? Yes or no, and why?
##Filter activity  DF to see if there are users with multiple purchases

many_convert = dfactivity.groupby('uid')['spent'].count()>1
many_convert


uid
1000039    False
1000045    False
1000071    False
1000101     True
1000123    False
           ...  
1049880    False
1049891    False
1049894    False
1049927    False
1049973    False
Name: spent, Length: 2094, dtype: bool

In [4]:
##What are the start and end dates of the experiment?

min_date =  dfgroups['join_dt'].min()
max_date =  dfactivity['dt'].max()

print("Start date:", min_date)
print("End date  :", max_date)






Start date: 2023-01-25
End date  : 2023-02-06


In [5]:
convert = dfactivity[dfactivity['spent'] >0].groupby(['uid'])['spent'].unique().count()
convert 


2094

In [6]:
## How many total users were in the experiment?
## How many users were in the control and treatment groups?

Tot_users = dfusers['id'].nunique()
A_users = dfgroups[dfgroups['group_name'] == 'A']['uid'].nunique()
B_users = dfgroups[dfgroups['group_name'] == 'B']['uid'].nunique()


print("Tot_users:", Tot_users)
print("A_users:", A_users)
print("B_users:", B_users)

Tot_users: 48943
A_users: 24343
B_users: 24600


In [7]:
## What was the conversion rate of all users?

converted = dfusers[dfusers['id'].isin(dfactivity['uid'].unique())]
conv_rate = converted.shape[0]/dfusers.shape[0]
print ("Conversion rate: ",round(100*conv_rate,2),"%")

Conversion rate:  4.28 %


In [8]:
## What is the user conversion rate for the control and treatment groups?

#### calculate total spent by group
tot_spent = dfactivity['spent'].sum()

A_activity = dfactivity[dfactivity['uid'].isin(dfgroups[dfgroups['group_name'] == 'A']['uid'])]
A_spent = A_activity['spent'].sum()


B_activity = dfactivity[dfactivity['uid'].isin(dfgroups[dfgroups['group_name'] == 'B']['uid'])]
B_spent = B_activity['spent'].sum()

#### calculate avgspent per group

A_avg_spent = A_spent/dfgroups[dfgroups['group_name'] == 'A'].shape[0]
B_avg_spent = B_spent/dfgroups[dfgroups['group_name'] == 'B'].shape[0]

print ("Spent by group A: ",round(A_spent,2),"$")
print ("Spent by group B: ",round(B_spent,2),"$")
print ("Average spent per user, group A: ",round(A_avg_spent,4),"$")
print ("Average spent per user, group B: ",round(B_avg_spent,4),"$")


Spent by group A:  82145.9 $
Spent by group B:  83415.33 $
Average spent per user, group A:  3.3745 $
Average spent per user, group B:  3.3909 $


#### We are to create a new table with all fields combined, plus count of purchases and total ammount.

In [9]:
### First we calculate total puchases per user and count of purchases

##user_spent = dfactivity.groupby('uid')['spent'].agg(['count', 'sum'])

##user_spent[min_date] = dfactivity[('dt')].min()    



user_spent = dfactivity.groupby('uid').agg({'spent': ['count', 'sum'], 'dt': 'min'})
user_spent.columns = ['count', 'sum', 'min_purchase_date']
user_spent['min_purchase_date'] = pd.to_datetime(user_spent['min_purchase_date'])
user_spent



,count,sum,min_purchase_date
uid,,,
1000039,1,36.650000,2023-02-04
1000045,1,51.580000,2023-01-30
1000071,1,6.710000,2023-02-02
1000101,2,23.804497,2023-02-02
1000123,1,100.740000,2023-01-26
...,...,...,...
1049880,1,186.690000,2023-01-26
1049891,1,47.140000,2023-01-26
1049894,1,34.050000,2023-01-25


In [10]:
## Now we merge all db and the aggregated values

dfmerged = dfusers.merge(dfgroups,left_on='id',right_on='uid',how='left')
dfmerged = dfmerged.drop('uid',axis=1)
dfmerged = dfmerged.merge(user_spent,left_on='id',right_on='uid',how='left')


##Finaly we sort it to be sure values are added correctly

dfmerged.sort_values('count', ascending=False)

,id,country,gender,group_name,join_dt,device,count,sum,min_purchase_date
42829,1043763,BRA,0,B,2023-02-03,A,2.0,21.997517,2023-02-04
19713,1020140,USA,F,A,2023-01-26,A,2.0,69.647605,2023-01-26
5551,1005676,MEX,F,A,2023-02-02,A,2.0,60.132379,2023-02-02
20670,1021115,TUR,F,B,2023-01-28,I,2.0,272.971545,2023-01-31
20594,1021037,ESP,M,B,2023-01-27,I,2.0,45.758676,2023-01-27
...,...,...,...,...,...,...,...,...,...
48938,1049995,BRA,F,B,2023-02-03,A,NaN,NaN,NaT
48939,1049996,USA,F,A,2023-01-29,A,NaN,NaN,NaT
48940,1049997,BRA,M,B,2023-02-03,A,NaN,NaN,NaT
48941,1049998,CAN,M,B,2023-02-03,I,NaN,NaN,NaT


In [11]:
###Fill Nan values with 0, create converted column for prportion calculations.

dfmerged['count'] = dfmerged['count'].fillna(0)
dfmerged['sum']= dfmerged['sum'].fillna(0)
dfmerged['converted'] = np.where(dfmerged['sum'] > 0, 1, 0)

dfmerged['join_dt'] = pd.to_datetime(dfmerged['join_dt'])
dfmerged['min_purchase_date'] = pd.to_datetime(dfmerged['min_purchase_date'])

dfmerged['days_diff'] = np.where(dfmerged['min_purchase_date'].notnull(), 
                                 (dfmerged['min_purchase_date']-dfmerged['join_dt']).dt.days, 0)

dfmerged = dfmerged.drop('min_purchase_date',axis=1)


                               
dfmerged.sort_values('count', ascending=False)


,id,country,gender,group_name,join_dt,device,count,sum,converted,days_diff
13045,1013329,MEX,0,B,2023-01-27,I,2.0,48.781381,1,0.0
45863,1046864,USA,M,A,2023-02-01,I,2.0,38.836688,1,0.0
46001,1047007,USA,F,B,2023-01-30,I,2.0,99.129562,1,1.0
17416,1017791,USA,M,B,2023-02-02,A,2.0,59.777310,1,0.0
27716,1028295,MEX,M,A,2023-01-31,I,2.0,234.409118,1,0.0
...,...,...,...,...,...,...,...,...,...,...
16663,1017014,USA,F,B,2023-01-25,A,0.0,0.000000,0,0.0
16664,1017015,0,M,B,2023-02-06,I,0.0,0.000000,0,0.0
16665,1017017,BRA,F,A,2023-01-30,I,0.0,0.000000,0,0.0
16666,1017018,GBR,M,A,2023-02-02,A,0.0,0.000000,0,0.0


In [12]:
dfmerged['novelty'] = np.where((dfmerged['days_diff'] == 0)&(dfmerged['sum']>0),1,0)
dfmerged.sort_values('novelty', ascending=False)


,id,country,gender,group_name,join_dt,device,count,sum,converted,days_diff,novelty
3441,1003519,GBR,M,B,2023-01-25,A,1.0,37.96,1,0.0,1
34368,1035112,MEX,F,B,2023-01-26,I,1.0,26.95,1,0.0,1
46943,1047965,MEX,M,A,2023-01-26,I,1.0,44.72,1,0.0,1
33003,1033706,GBR,M,A,2023-02-01,I,1.0,98.17,1,0.0,1
40559,1041445,AUS,F,B,2023-01-27,A,1.0,153.55,1,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...
16595,1016943,USA,M,A,2023-01-26,I,0.0,0.00,0,0.0,0
16596,1016944,USA,M,A,2023-01-29,I,0.0,0.00,0,0.0,0
16597,1016945,USA,F,A,2023-01-26,A,0.0,0.00,0,0.0,0
16598,1016946,BRA,F,B,2023-01-29,A,0.0,0.00,0,0.0,0


In [13]:
dfmerged.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48943 entries, 0 to 48942
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   id          48943 non-null  int64         
 1   country     48943 non-null  object        
 2   gender      48943 non-null  object        
 3   group_name  48943 non-null  object        
 4   join_dt     48943 non-null  datetime64[ns]
 5   device      48943 non-null  object        
 6   count       48943 non-null  float64       
 7   sum         48943 non-null  float64       
 8   converted   48943 non-null  int32         
 9   days_diff   48943 non-null  float64       
 10  novelty     48943 non-null  int32         
dtypes: datetime64[ns](1), float64(3), int32(2), int64(1), object(4)
memory usage: 3.7+ MB


In [14]:
##Last step, we export it!!

dfmerged.to_csv('D:\DATA ANALYSIS\Final_proj\GOblo.csv')

In [ ]:
### END file